# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

In [61]:
import pandas as pd
import numpy as np
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [62]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

#### Next, from Module 2 (Multiple Regression), copy and paste the ‘get_numpy_data’ function (or equivalent) that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function returns a ‘feature_matrix’ (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ which is an array of the values of the output in the data set (e.g. ‘price’).

In [63]:
def get_numpy_data(dataframe, features, output):
    dataframe['constant'] = 1 # add a constant column to an DFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of dataFrame given by the ‘features’ list into the DFrame ‘features_df’
    features_df = dataframe[features]
    # this will convert the features_df into a numpy matrix:
    features_matrix = features_df.to_numpy()
    # assign the column of dataframe associated with the target to the variable ‘output_sarray’
    output_sarray = dataframe[output]
    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy()
    return(features_matrix, output_array)

#### Similarly, copy and paste the ‘predict_output’ function (or equivalent) from Module 2. This function accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and return a 1D array ‘predictions’.

In [64]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [65]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [66]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [67]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [68]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return (normalized_features, norms)

In [69]:
#test out function

features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [70]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [71]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [72]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [73]:
prediction = predict_outcome(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [74]:
ro=[0,0,0]
for i in range(simple_feature_matrix.shape[1]):
    ro[i]= ([simple_feature_matrix[:,i]]*(output - prediction + weights[i]*simple_feature_matrix[:,i])).sum()

In [75]:
ro

[79400300.0145229, 87939470.82325175, 80966698.66623947]

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

we have ro[0], ro[1], ro[2]

For W1 to be zero, we need ro[1] in [-lambda/2, lambda/2]

We have -lambda/2 <= ro[1] <= lambda/2

This translates to lambda >= -2ro[1] and lambda >= 2ro[1]

For both conditions to be satisfied, lambda >= 2ro[1] = 1.75e8

Similarly for W2. lambda >= 2ro[2] = 1.62e8.

So, w[i] = 0 if lambda >= 2 * abs(ro[i])

For quiz 1:

What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

Simply calculate the value for lambda that will set w2 to zero

i.e. a lambda value that is greater than 2*ro[2]

but that is less than the value that will set w1 to zero

i.e. a lambda value that is less then 2*ro[1]

This will be a range of lambda values between 2ro[2] and 2ro[1]

In [76]:
print(2*ro[1])
print(2*ro[2])

175878941.6465035
161933397.33247894


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

**i.e. a lambda value that is greater than both 2ro[1] and 2ro[2] (in this case just 2*ro[1] as this is the greater)**

So we can say that ro[i] quantifies the significance of the i-th feature: the larger ro[i] is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [77]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_outcome(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i =  (feature_matrix[:,i]*(output - prediction + weights[i]*feature_matrix[:,i])).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return (new_weight_i)

To test the function, run the following cell:

In [78]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [79]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.array(initial_weights)
    change = np.zeros(len(weights))
    converged = False

    while not converged:

        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
        
            # use old_weights_i to compute change in coordinate
            change[i] = np.abs(old_weights_i - weights[i])
            
        max_change = max(change)
        
        if max_change < tolerance:
            converged = True
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [80]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [81]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [82]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [83]:
#RSS of the learned model 
prediction = predict_outcome(normalized_simple_feature_matrix, weights)
print("RSS: ",np.sum((prediction-output)**2))

#print that had zero weight at convergence
weights

RSS:  1630492476715386.5


array([21624997.95951909, 63157247.20788956,        0.        ])

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [95]:
train_data = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
train_data['floors'] = train_data['floors'].astype(float) 
test_data['floors'] = test_data['floors'].astype(float) 

Let us consider the following set of features.

In [85]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [96]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, norms) = normalize_features(all_feature_matrix) # normalize features

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [98]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [137]:
for i in range(len(all_features)):
    print(all_features[i],':',weights1e7[i+1]) #ignore the added constant weight in the first column
    
#include the constant:
print('constant:',weights1e7[0])

bedrooms : 0.0
bathrooms : 0.0
sqft_living : 48389174.771548964
sqft_lot : 0.0
floors : 0.0
waterfront : 3317511.2149216533
view : 7329961.811714254
condition : 0.0
grade : 0.0
sqft_above : 0.0
sqft_basement : 0.0
yr_built : 0.0
yr_renovated : 0.0
constant: 24429600.23440312


Next, learn the weights with l1_penalty=1e8, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights weights1e8, you will need them later.

In [113]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [117]:
weights1e8

array([71114625.71488702,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [139]:
for i in range(len(all_features)
              ):
    print(all_features[i],':',weights1e8[i+1]) #ignore the added constant weight in the first column
    
#include the constant:
print('constant:',weights1e8[0])

bedrooms : 0.0
bathrooms : 0.0
sqft_living : 0.0
sqft_lot : 0.0
floors : 0.0
waterfront : 0.0
view : 0.0
condition : 0.0
grade : 0.0
sqft_above : 0.0
sqft_basement : 0.0
yr_built : 0.0
yr_renovated : 0.0
constant: 71114625.71488702


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [141]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

for i in range(len(all_features)
              ):
    print(all_features[i],':',weights1e4[i+1]) #ignore the added constant weight in the first column
    
#include the constant:
print('constant:',weights1e4[0])

bedrooms : -22097398.92430532
bathrooms : 12791071.872785171
sqft_living : 93808088.09281193
sqft_lot : -2013172.7570495396
floors : -4219184.932650143
waterfront : 6482842.817535064
view : 7127408.534806886
condition : 5001664.854696402
grade : 14327518.437140508
sqft_above : -15770959.152373968
sqft_basement : -5159591.22213147
yr_built : -84495341.7684364
yr_renovated : 2824439.4970368347
constant: 78564738.34156762


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [121]:
def normalized_weight(weights, features):
    features, norms = normalize_features(features)
    weights_normalized = weights/norms
    return weights_normalized

In [131]:
normal_weights =[]
for w in [weights1e4,weights1e7,weights1e8]:
    normal_weights.append(normalized_weight(w,all_feature_matrix))

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [126]:
normalized_weights1e7 = normalized_weight(weights1e7,all_feature_matrix)
normalized_weights1e7[3]

161.31745764611762

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [128]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [136]:
for weight in normal_weights:
    prediction = predict_outcome(test_feature_matrix, weight)
    print("RSS:",np.sum((prediction-test_output)**2))
    #THE ORDER IS weights1e4,weights1e7,weights1e8

RSS: 228459958971393.25
RSS: 275962075920366.78
RSS: 537166151497322.75


***QUIZ QUESTION***

Which model performed best on the test data?


**ANS:** weights1e4 model has the lowest RSS (best)